# Deploy a pretrained sentiment model from Watson NLP as a Python function

This notebook shows the steps needed to create and deploy a Python function that predicts sentiment in Watson Machine Learning. the example creates an _online deployment_ of the Python function. This way, the sentiment analysis can be invoked via a web service in applications.

## Prerequisites 

Before running this notebook: Create a new deployment space, or identify an existing deployment space to use for this example

## Initialize the Watson Machine Learning client

In [1]:
CPD_URL = '<CPD PLATFORM URL>'

In [2]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [3]:
wml_credentials = {
    "url": CPD_URL,
    "token": wslib.auth.get_current_token(),
    "instance_id": "wml_local",
    "version" : "4.6"
}

In [4]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

In [ ]:
client.spaces.list()

### Add the `ID` of the space to use for deploying the function

In [7]:
client.set.default_space('<SPACE ID>')

'SUCCESS'

## Create a Python function that detects sentiment

This function is using the targeted sentiment block `targets-sentiment_sequence-bert_multi_stock` provided by Watson NLP. This model requires the syntax block `syntax_izumo_en_stock` for preprocessing. Hence, the function applies the syntax block on the incoming text before invoking the sentiment block.

In [8]:
def detect_sentiment():
    import watson_nlp
    
    syntax_model = watson_nlp.load('syntax_izumo_en_stock')
    sentiment_model = targets_sentiment_model = watson_nlp.load('targets-sentiment_sequence-bert_multi_stock')
    
    def score(input):
        scoring_prediction_out = []
        for input_data_row in input["input_data"][0]["values"]: 
            scoring_prediction_row = []
            for input_data in input_data_row:            
                syntax_result = syntax_model.run(input_data, 
                                                     parsers=('token', 'lemma', 'part_of_speech'))
                targets_sentiments = targets_sentiment_model.run(syntax_result)
                scoring_prediction_row.append(targets_sentiments.to_dict())
                
            scoring_prediction_out.append(scoring_prediction_row)
            
        # Score using the pre-defined model
        scoring_response = {
            'predictions': [{'fields': ['nlp_prediction'], 
                             'values': scoring_prediction_out
                            }]
        }

        return scoring_response
    
    return score

## Save the Python function to the Model repository

In [11]:
pyfunc_swspec_id = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")

meta_data = {
    client.repository.FunctionMetaNames.NAME: 'sentiment_function',
    client.repository.FunctionMetaNames.DESCRIPTION: 'sentiment_function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: pyfunc_swspec_id
}


sentiment_function_details = client.repository.store_function(meta_props=meta_data, function=detect_sentiment)

## Create an Online Deployment

In [ ]:
sentiment_function_uid = client.repository.get_function_uid(sentiment_function_details)

meta_props = {
   client.deployments.ConfigurationMetaNames.NAME: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.DESCRIPTION: "nlp-sentiment-deploy",
   client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
   client.deployments.ConfigurationMetaNames.ONLINE: {   }
}

sentiment_deployment_details = client.deployments.create(sentiment_function_uid, meta_props=meta_props)

sentiment_deployment_id = client.deployments.get_uid(sentiment_deployment_details)



#######################################################################################

Synchronous deployment creation for uid: '55468b7e-b695-4337-b3bd-234f76121136' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......................................................

## Predict sentiment of sample data

In [ ]:
scoring_payload = {
   client.deployments.ScoringMetaNames.INPUT_DATA: [{  
        'fields': ["Text"],
        'values': [["The room is nice, but the price is too expensive. The window frame is made of wood."],
                   ["The check-in was horrible."],
                  ]
   }]
}

In [ ]:
predictions = client.deployments.score(sentiment_deployment_id, scoring_payload)
predictions

## Clean up: Delete the deployment and the function

In [ ]:
client.deployments.delete(sentiment_deployment_id)

In [ ]:
client.repository.delete(sentiment_function_uid)